In [ ]:
from core.signals import VIX


vix = VIX.from_fred_data()
vix.data

In [4]:
from pkg.src.core import data


gdp_tickers = {
    "GDPC1" : "RealGDP",
}
gdp_comp_tickers = {
    "PCECC96": "Consumption",
    "GCEC1": "GovSpending",
    "GPDIC1": "Investments",
    "NETEXC": "NetExports",
}

GDP = data.get_macro(list(gdp_tickers.keys()))
GDP = GDP.rename(columns=gdp_tickers)

gdp_components = data.get_macro(list(gdp_comp_tickers.keys())).loc["2019":]
gdp_components = gdp_components.rename(columns=gdp_comp_tickers)

import plotly.graph_objects as go

# Create a figure object
fig = go.Figure()

# Add trace for each column in the DataFrame
for column in gdp_components.columns:
    fig.add_trace(go.Bar(
        x=gdp_components.index,
        y=gdp_components[column],
        name=column
    ))

# Update layout
fig.update_layout(
    barmode='stack',
    xaxis_title='Date',
    yaxis_title='Consumption'
)

# Show the plot
fig.show()


In [7]:
yield_curve = {
    "T10Y2Y" : "10Y-2Y",
    "T10Y3M" : "10Y-3M"
}

yield_curve_data = data.get_macro(list(yield_curve.keys())).loc["2019":]
yield_curve_data = yield_curve_data.rename(columns=yield_curve)
yield_curve_data.head().to_markdown()

'| DATE                |   10Y-2Y |   10Y-3M |\n|:--------------------|---------:|---------:|\n| 2019-01-01 00:00:00 |   nan    |   nan    |\n| 2019-01-02 00:00:00 |     0.16 |     0.24 |\n| 2019-01-03 00:00:00 |     0.17 |     0.15 |\n| 2019-01-04 00:00:00 |     0.17 |     0.25 |\n| 2019-01-07 00:00:00 |     0.17 |     0.25 |'

In [8]:
import plotly_express as px

px.line(yield_curve_data)

In [ ]:
my_list = [10, 20, 30, 40, 50]

# Get the index of a value in the list
value = 30

if value in my_list:
    index = my_list.index(value)
    print("Index of", value, "in the list:", index)
else:
    print(value, "is not present in the list.")


In [ ]:
from core import metrics

metrics.to_hurst_exponent(vix.data.rolling(20).apply(metrics.to_momentum, kwargs={"months":1}), lags=20)


In [ ]:
import json
# from core.signals import OECDUSLEIHP
from pkg.core.strategies import BacktestManager

# signal = OECDUSLEIHP.from_fred_data()

def get_universe(name: str = "USSECTOR"):
    with open("universe.json") as f:
        return json.load(f).get(name)
    
from pkg.src.core import data

prices = data.get_prices(tickers = list(get_universe().keys()))
bt = BacktestManager(prices=prices, start="2007-1-1", commission=10, shares_frac=0)
# bt.Momentum(target_percentile=0.7, months=3)
# bt.values.plot(figsize=(12, 10))
# bt.analytics

from core import metrics

metrics.to_momentum(prices=prices.iloc[:, 0], months=12, skip_months=1)


In [ ]:

import json
import numpy as np
from pkg.src.core.portfolios import Optimizer
from pkg.src.core import data
from pkg.src.core import metrics
from scipy.cluster.hierarchy import linkage, to_tree
from scipy.spatial.distance import squareform



def get_universe(name: str = "USSECTOR"):
    with open("pkg/src/universe.json") as f:
        return json.load(f).get(name)
    
from pkg.src.core import data

prices = data.get_prices(tickers = list(get_universe().keys()))




In [ ]:
opt = Optimizer.from_prices(prices=prices.loc["2017"].dropna(axis=1))

w = opt.hierarchical_risk_parity()

w

In [ ]:

from pkg.src.core.portfolios import objectives

np.sum(
    objectives.risk_contributions(
        weights=w, covariance_matrix=np.array(opt.covariance_matrix),
        sub_covariance_matrix_idx=[8]
    )
)

np.sum(
    objectives.risk_contributions(
        weights=w, covariance_matrix=np.array(opt.covariance_matrix),
        sub_covariance_matrix_idx=[0, 3]
    )
)


In [ ]:
objectives.expected_volatility(w, opt.covariance_matrix) 

In [ ]:

dist = np.sqrt((1 - opt.correlation_matrix).round(5) / 2)
clusters = linkage(squareform(dist), method="single")
sorted_tree = list(to_tree(clusters, rd=False).pre_order())

print(sorted_tree)
def recursive_bisection(sorted_tree):
    """_summary_

    Args:
        sorted_tree (_type_): _description_

    Returns:
        List[Tuple[List[int], List[int]]]: _description_
    """

    left = sorted_tree[0 : int(len(sorted_tree) / 2)]
    right = sorted_tree[int(len(sorted_tree) / 2) :]
    
    if len(sorted_tree) < 3:
        return (left, right)
    
    cache = [(left, right)]
    if len(left) > 2:
        cache.extend(recursive_bisection(left))
    if len(right) > 2:
        cache.extend(recursive_bisection(right))
    return cache
    

cluster_sets = recursive_bisection(sorted_tree)

for i in cluster_sets:
    
    print(i)

In [ ]:
from core import data
import plotly.graph_objects as go
from plotly.subplots import make_subplots

tickers = {
    "T5YIE": "Breakeven Inflation: 5Y",
    "T10YIE": "Breakeven Inflation: 10Y",
    "T5YIFR": "Expectation Infaltiona: 5Y",
}

result = data.get_macro(tickers=tickers).dropna()
result["chg"] = result["Breakeven Inflation: 10Y"] - result["Breakeven Inflation: 5Y"]
spy = data.get_prices(tickers="SPY")

# Create a line plot for each series
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)
fig.add_trace(
    go.Scatter(x=spy.index, y=spy["SPY"], mode="lines", name="SPY"), row=1, col=1
)
for series in result:
    fig.add_trace(
        go.Scatter(x=result.index, y=result[series], mode="lines", name=series),
        row=2,
        col=1,
    )

# Set the title and axis labels
fig.update_layout(title="Inflation", xaxis_title="Date", yaxis_title="%", height=600)
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1d", step="day", stepmode="backward"),
                    dict(count=7, label="1w", step="day", stepmode="backward"),
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=False),
        type="date",
    ),
    hovermode="x unified",  # Enable the "Show All Stats" option on mouseover
)

# Display the graph
fig.show()

In [ ]:
from core.strategies import BacktestManager
bt = BacktestManager.from_universe(start="2010-1-1", commission=10, shares_frac=0)
bt.Momentum()
bt.values.plot()
